In [265]:
import pandas as pd
import os
from ast import literal_eval

In [259]:
songs_composition = pd.read_csv('data/songs_composition.csv')

In [260]:
def df_get_id(filenames):
    ids = []
    for filename in filenames:
        id_ = filename[0].split('/')[-1].split('.')[0]
        ids.append(id_)
    return ids

def df_get_topics(df_topics):
    list_topics = []
    for row in df_topics:
        topics = {}
        for i in range(20):
            topics[i] = row[i + 1]
        list_topics.append(topics)
    return list_topics

In [261]:
songs_composition['track_id'] = df_get_id(songs_composition.values)
songs_composition['topics'] = df_get_topics(songs_composition.values)
data_songs = pd.read_csv('data/data_lyrics.csv')

In [262]:
id_topics = songs_composition[['track_id', 'topics']]

result = pd.merge(id_topics, data_songs, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

genres = result['genres'].values
genres = set(genres)

topics_genres = {}
topics_all = {}


for index, row in result.iterrows():
    genre = row['genres']
    topics = row['topics']
    year = row['year']
    
    if genre not in  topics_genres:
        topics_genres[genre] = {}
    
    for topic in topics:
        if not topic in topics_genres[genre]:
            topics_genres[genre][topic] = {}
        if not year in topics_genres[genre][topic]:
            topics_genres[genre][topic][year] = []
            topics_genres[genre][topic][year].append(0)
            topics_genres[genre][topic][year].append(0)
        topics_genres[genre][topic][year][0] += topics[topic]
        topics_genres[genre][topic][year][1] += 1
        
        if topic not in topics_all:
            topics_all[topic] = {}
        
        if not year in topics_all[topic]:
            topics_all[topic][year] = []
            topics_all[topic][year].append(0)
            topics_all[topic][year].append(0)
        topics_all[topic][year][0] += topics[topic]
        topics_all[topic][year][1] += 1

Now that we have the corresponding topics coefficients for a given song id, we add them to the all dataframe.

In [263]:
results = {}
for genre in topics_genres:
    results[genre] = {}
    for topic in topics_genres[genre]:
        results[genre][topic] = {}
        for year in topics_genres[genre][topic]:
            if year:
                mean = topics_genres[genre][topic][year][0] / topics_genres[genre][topic][year][1] 
                results[genre][topic][year] = mean
results

results_all = {}
for topic in topics_all:
    results_all[topic] = {}
    for year in topics_all[topic]:
        if year:
            mean = topics_all[topic][year][0] / topics_all[topic][year][1] 
            results_all[topic][year] = mean

{0: {1954: 0.00028801650017799997,
  1956: 0.000279745852912,
  1963: 0.0002320234458545,
  1966: 0.00010967597592,
  1967: 0.000136584052311,
  1968: 0.00016968481398870456,
  1969: 0.00034472951211525005,
  1971: 0.0001984027626025,
  1972: 0.00025583387209119805,
  1973: 0.00013586673055596756,
  1974: 0.0001337824545205,
  1975: 0.029852877530931,
  1976: 0.00014532160754994082,
  1977: 0.00018763069073071434,
  1979: 0.2966148091798546,
  1980: 0.0002874179803478093,
  1981: 0.00014516731514653785,
  1982: 0.00016280799256274998,
  1983: 0.000135635170944,
  1984: 0.0001539644294360975,
  1985: 0.00020026321757785707,
  1986: 0.000145783867416,
  1987: 0.010837385143680161,
  1988: 0.0003579856050622,
  1989: 0.00018586970804558082,
  1990: 0.0015766008904194288,
  1991: 0.0002210106750039506,
  1992: 0.0001512620367937482,
  1993: 0.0001504248967475,
  1994: 0.00014231797068521985,
  1995: 0.00013959488202130633,
  1996: 0.00023134665135012212,
  1997: 0.00018641853074549997,
  1

In [264]:
path = 'data/topics/genres'
for genre in results:
    for topic in results[genre]:
        directory = path + '/' + genre + '/'
        if not os.path.exists(directory):
            os.makedirs(directory)
        csvfile = open((directory + str(topic) + '.csv'), 'w')
        #for year in results[genre][topic]:
        for year in sorted(results[genre][topic].keys()):
            csvfile.write(str(year) + ', ' + str(results[genre][topic][year]) + '\n')
        csvfile.close()

path = 'data/topics/genres'
directory = path + '/' + 'all' + '/'
if not os.path.exists(directory):
    os.makedirs(directory)
    
for topic in results_all:
    csvfile = open((directory + str(topic) + '.csv'), 'w')
    #for year in results[genre][topic]:
    for year in sorted(results_all[topic].keys()):
        csvfile.write(str(year) + ', ' + str(results_all[topic][year]) + '\n')
    csvfile.close()